In [1]:
#Intall necessary modules and import required packages
import nltk
import random
import json
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
#To parse the JSON file into python we used JSON package
data = open('intents.json').read()
intents = json.loads(data)
all_words=[]
all_classes = []
all_documents = []
ignore_words = ['?', '!']

In [3]:
def processing_data(pattern):
   #fetch patterns and tokenize them into words
   self.pattern=list(map(lambda x:x["patterns"],self.intents))
   self.words=list(map(word_tokenize,flatten(self.pattern)))

In [4]:
#preprocessing the data
#before making a model we pwerformed various preprocessing techniques
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #By fetching the patters we tokenized the data: using nltk.word_tokenize().
        wd_tokens = nltk.word_tokenize(pattern)
        all_words.extend(wd_tokens)
        #append words in the word list
        all_documents.append((wd_tokens, intent['tag']))

        #we created list of classes  for tags
        if intent['tag'] not in all_classes:
            all_classes.append(intent['tag'])

In [5]:
nltk.download('omw-1.4')
# We lemmatized and lowered each word and removed duplicates by creating a pickle file
all_words = [lemmatizer.lemmatize(wd_tokens.lower()) for wd_tokens in all_words if wd_tokens not in ignore_words]
all_words = sorted(list(set(all_words)))
# all classes are sorted
all_classes = sorted(list(set(all_classes)))
# documents is combination between intents and patterns
print (len(all_documents), "all_documents")
# now classes = intents
print (len(all_classes), "all_classes", all_classes)
# words is all words and vocabulary
print (len(all_words), "unique lemmatized words", all_words)


pickle.dump(all_words,open('all_words.pkl','wb'))
pickle.dump(all_classes,open('all_classes.pkl','wb'))


[nltk_data] Downloading package omw-1.4 to C:\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


379 all_documents
58 all_classes ['ABM procedures', 'Academic calendar', 'Admissions', 'Alumni', 'CAAS Staff', 'CAAS services', 'CECIL C. HUMPHREYS SCHOOL OF LAW', 'Canvas', 'Combination Senior Program Guide', 'Contact number', 'Course', 'Current students', 'Explore UofM', 'Faculty and staff', 'Fees', 'Grades', 'Graduate Certificate Programs', 'Graduate Non-Degree Status', 'Graduate catalog', 'Holds', 'Hours of operation', 'International admissions', 'Libraries', 'Location', 'Mymemphis', 'Nationally Ranked Graduate Programs', 'News and events', 'Registration', 'Reverse transfer', 'Transcripts', 'University ID', 'University Residence', 'about', 'academic advisor', 'admission deadlines', 'apply to UofM', 'attendance', 'center for athletic academic services', 'commencement', 'complaint', 'contact graduation analyst', 'degree programs', 'freashman application fee waiver', 'goodbye', 'greeting', 'help', 'law admissions', 'name', 'personal data', 'readmission', 'scholarships', 'self service 

In [6]:
# We created training and test data
training_data = []
# We created empty array for output
empty_array = [0] * len(all_classes)
# for each sentence we initialized bag of words and training set
for doc in all_documents:
    # we initialized bag of words
    bag = []
    # For the pattern - list of tokenized words
    pattern_words = doc[0]
    # To represent related words-  lemmatize each word and create base word, 
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # we created bag of words array with 1, if word found in the current pattern
    for wd_tokens in all_words:
        bag.append(1) if wd_tokens in pattern_words else bag.append(0)
    
    # The output will be '0' for each tag and '1' for current tag for each pattern
    output_row = list(empty_array)
    output_row[all_classes.index(doc[1])] = 1
    
    training_data.append([bag, output_row])
# we shuffled features and turned them into np.array
random.shuffle(training_data)
training_data = np.array(training_data)
# and then we created train and test lists. X as patterns, Y as intents
train_x = list(training_data[:,0])
train_y = list(training_data[:,1])
print("created training data")

#Model building

# we created a model which is a deep neural network with three layers. First layer has 128 neurons, second layer has 64 neurons and third is output layer which containes contains number of neurons
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model. This model give good results.
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#We finally fit and saved the modelas chatbot_model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('uofmchatbot_model', hist)

print("model is created and saved as uofmchatbot_model")

c:\Users\Shinestar\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


created training data


c:\Users\Shinestar\.conda\envs\tensorflow\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
76/76 [==============================] - 3s 5ms/step - loss: 4.0506 - accuracy: 0.0264
Epoch 2/200
76/76 [==============================] - 0s 4ms/step - loss: 3.9014 - accuracy: 0.0633
Epoch 3/200
76/76 [==============================] - 0s 4ms/step - loss: 3.7551 - accuracy: 0.1003
Epoch 4/200
76/76 [==============================] - 0s 4ms/step - loss: 3.5779 - accuracy: 0.1451
Epoch 5/200
76/76 [==============================] - 0s 4ms/step - loss: 3.3715 - accuracy: 0.2084
Epoch 6/200
76/76 [==============================] - 0s 4ms/step - loss: 3.1847 - accuracy: 0.2269
Epoch 7/200
76/76 [==============================] - 0s 4ms/step - loss: 2.8977 - accuracy: 0.2718
Epoch 8/200
76/76 [==============================] - 0s 6ms/step - loss: 2.7092 - accuracy: 0.3272
Epoch 9/200
76/76 [==============================] - 0s 4ms/step - loss: 2.6085 - accuracy: 0.3351
Epoch 10/200
76/76 [==============================] - 0s 6ms/step - loss: 2.3442 - accuracy: 0.3958
Epoch 11/